In [1]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'


import findspark
findspark.init()
findspark.find()

'/usr/lib/spark/'

In [16]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("yarn") \
        .config("spark.driver.cores", "2") \
        .config("spark.driver.memory", "2g") \
        .appName("myapp2") \
        .getOrCreate()

print(spark)

# данные  датафрейма 
data = [('2021-01-06', 3744, 63, 322),
        ('2021-01-04', 2434, 21, 382),
        ('2021-01-04', 2434, 32, 159),
        ('2021-01-05', 3744, 32, 159),
        ('2021-01-06', 4342, 32, 159),
        ('2021-01-05', 4342, 12, 259),
        ('2021-01-06', 5677, 12, 259),
        ('2021-01-04', 5677, 23, 499)
]
# названия атрибутов
columns = ['dt', 'user_id', 'product_id', 'purchase_amount']
# создаём датафрейм
df = spark.createDataFrame(data=data, schema=columns)

In [19]:
window = Window().partitionBy('user_id')
df_window_agg= df.withColumn("max",F.max("purchase_amount").over(window)) \
                 .withColumn("min",F.min("purchase_amount").over(window))
df_window_agg.select('user_id','max','min').distinct().show()

+-------+---+---+
|user_id|max|min|
+-------+---+---+
|   2434|382|159|
|   4342|259|159|
|   5677|499|259|
|   3744|322|159|
+-------+---+---+



In [10]:
# импортируем оконную функцию и модуль Spark Functions
# from pyspark.sql.window import Window 
from pyspark.sql import Window, types
import pyspark.sql.functions as F

# создаём объект оконной функции
window = Window().partitionBy(['dt']).orderBy(F.asc('purchase_amount'))

# создаём колонку с рассчитанной статистикой по оконной функции
df_window = df.withColumn("rank", F.rank().over(window))

# выводим нужные колонки
df_window.select('dt', 'user_id', 'rank', 'purchase_amount').show()

+----------+-------+----+---------------+
|        dt|user_id|rank|purchase_amount|
+----------+-------+----+---------------+
|2021-01-06|   4342|   1|            159|
|2021-01-06|   5677|   2|            259|
|2021-01-06|   3744|   3|            322|
|2021-01-05|   3744|   1|            159|
|2021-01-05|   4342|   2|            259|
|2021-01-04|   2434|   1|            159|
|2021-01-04|   2434|   2|            382|
|2021-01-04|   5677|   3|            499|
+----------+-------+----+---------------+



In [14]:
# from pyspark.sql.functions import dense_rank
window2 = Window().orderBy(F.asc('purchase_amount'))
df_window2 = df.withColumn("row_number", F.row_number().over(window2))
df_window2.select('dt', 'user_id', 'purchase_amount', 'row_number').show()

2023-08-08 20:38:34,062 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+-------+---------------+----------+
|        dt|user_id|purchase_amount|row_number|
+----------+-------+---------------+----------+
|2021-01-04|   2434|            159|         1|
|2021-01-05|   3744|            159|         2|
|2021-01-06|   4342|            159|         3|
|2021-01-05|   4342|            259|         4|
|2021-01-06|   5677|            259|         5|
|2021-01-06|   3744|            322|         6|
|2021-01-04|   2434|            382|         7|
|2021-01-04|   5677|            499|         8|
+----------+-------+---------------+----------+



In [15]:
# импортируем оконную функцию и модуль Spark Functions
from pyspark.sql.window import Window 
import pyspark.sql.functions as F

# создаём объект оконной функции
window = Window().partitionBy('user_id').orderBy('dt')

# создаём колонку с рассчитанной статистикой по оконной функции
dfWithLag = df.withColumn("lag_1",F.lag("purchase_amount", 1).over(window))

# фильтруем данные
dfWithLag .select('dt','user_id', 'purchase_amount','lag_1').show()

+----------+-------+---------------+-----+
|        dt|user_id|purchase_amount|lag_1|
+----------+-------+---------------+-----+
|2021-01-04|   2434|            382| null|
|2021-01-04|   2434|            159|  382|
|2021-01-05|   4342|            259| null|
|2021-01-06|   4342|            159|  259|
|2021-01-04|   5677|            499| null|
|2021-01-06|   5677|            259|  499|
|2021-01-05|   3744|            159| null|
|2021-01-06|   3744|            322|  159|
+----------+-------+---------------+-----+

